# <a id='toc1_'></a>[Modélisation des mobilités quotidiennes des individus](#toc0_)

Le but de ce notebook est d'explorer une méthodologie pour prédire les séquences de déplacements d'un individu à partir de ses caratéristiques socio-économiques.  
Ces premiers travaux ont pour objectifs de qualifier la pertinence de l'utilisation des algorithmes du traitement du langage naturel pour cette tâche.  


Définitions : 
- **Déplacement** : un déplacement est défini par un triplet (mode de déplacement, motif de déplacement, distance parcourue)
- **Séquence de déplacements** : c'est une suite ordonnée de déplacement effectué par un individu sur une journée.

Cette métohodologie se divise en 2 parties :
1. Apprentissage d'un modèle d'embedding pour effectuer un plongement sémentique des couples (mode, motif)
2. Apprentissage d'un RNN "vector to sequence" pour la prédiction d'une séquence de déplacement à partir d'un vecteur de caratistique socio-économique 

**Sommaire**<a id='toc0_'></a>    
- [Création dataset](#toc1_1_)    
  - [Récupération Data from mobility](#toc1_1_1_)    
  - [Préprocessing des données](#toc1_1_2_)    
- [Embedding des couples (MODE, MOTIF)](#toc1_2_)    
  - [Description du modèle](#toc1_2_1_)    
  - [Entrainement du modèle](#toc1_2_2_)    
  - [Data viz des 2 premières dimensions de l'emmbedings](#toc1_2_3_)    
- [RNN vector to Sequence](#toc1_3_)    
  - [Description du modèle](#toc1_3_1_)    
  - [Entrainement du modèle](#toc1_3_2_)    
  - [Exemple de prédiction](#toc1_3_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import plotly.express as px
from core import DataGetter, Mob2Vec, Features2Trajectory

## <a id='toc1_1_'></a>[Création dataset](#toc0_)

### <a id='toc1_1_1_'></a>[Récupération des données](#toc0_)
Les données utilisées sont issues de l'Enquète Mobilité des Personnes 2019 (EMP). Elles sont récupérées via le package mobility.

In [2]:
data = DataGetter(n_individus=10000, group_modalities=True)
data.get_features()
data.get_trajectories()

c:\Users\Quentin\Desktop\ign\core\data_getter.py:73: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  self.trajectories.groupby(self.column_id)[


### <a id='toc1_1_2_'></a>[Préprocessing des données](#toc0_)

In [3]:
data.pad_trajectories()
data.distance_standardization()

## <a id='toc1_2_'></a>[Embedding des couples (MODE, MOTIF)](#toc0_)

L'objectif de ce modèle est d'apprendre un plongement sémentique de notre vocabulaire de déplacements.

Nous apprenons un modèle équivalent à word2vec : "un déplacement, définit par le couple (motif, mode), est à une trajectoire, ce qu'un mot est à une phrase"

### <a id='toc1_2_1_'></a>[Description du modèle](#toc0_)

In [4]:
mob2vec = Mob2Vec(data, window_size=2, embedding_dim=2)
mob2vec.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 2)              164       
                                                                 
 global_average_pooling1d (G  (None, 2)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 82)                246       
                                                                 
Total params: 410
Trainable params: 410
Non-trainable params: 0
_________________________________________________________________


### <a id='toc1_2_2_'></a>[Entrainement du modèle](#toc0_)

In [5]:
mob2vec.fit()

Epoch 1/20
98/98 [==============================] - 1s 3ms/step - loss: 4.3095 - val_loss: 4.1958
Epoch 2/20
98/98 [==============================] - 0s 2ms/step - loss: 4.0538 - val_loss: 3.8837
Epoch 3/20
98/98 [==============================] - 0s 2ms/step - loss: 3.7023 - val_loss: 3.4977
Epoch 4/20
98/98 [==============================] - 0s 2ms/step - loss: 3.3110 - val_loss: 3.1086
Epoch 5/20
98/98 [==============================] - 0s 2ms/step - loss: 2.9501 - val_loss: 2.7805
Epoch 6/20
98/98 [==============================] - 0s 2ms/step - loss: 2.6720 - val_loss: 2.5496
Epoch 7/20
98/98 [==============================] - 0s 2ms/step - loss: 2.4915 - val_loss: 2.4123
Epoch 8/20
98/98 [==============================] - 0s 2ms/step - loss: 2.3910 - val_loss: 2.3392
Epoch 9/20
98/98 [==============================] - 0s 2ms/step - loss: 2.3372 - val_loss: 2.2989
Epoch 10/20
98/98 [==============================] - 0s 2ms/step - loss: 2.3054 - val_loss: 2.2733
Epoch 11/20
98/98 [

In [6]:
px.line(pd.DataFrame(mob2vec.history.history))

### <a id='toc1_2_3_'></a>[Data viz des 2 premières dimensions de l'emmbedings](#toc0_)

In [7]:
df_embedding = pd.DataFrame(columns=["Mode", "Motif", "x", "y"])
df_embedding[["x", "y"]] = mob2vec.embedding_layer.embeddings[:, :2]
df_embedding[["Mode", "Motif"]] = list(data.vocab.keys())

px.scatter(df_embedding, x="x", y="y", color="Motif", symbol="Mode")

## <a id='toc1_3_'></a>[RNN vector to Sequence](#toc0_)

### <a id='toc1_3_1_'></a>[Description du modèle](#toc0_)

Instanciation d'un RNN "vector to sequence".

Input : données scoio économique d'un individu (csp, nb de voitures, type de commune) à l'instant t=0, et 0 à l'instant t>0.  
Output : séquence de déplacements sur une journée. Un déplacement est défini par le triplet (motif, mode, distance).


In [8]:
feats2traj = Features2Trajectory(data, mob2vec)
feats2traj.get_trajectories_for_rnn()
feats2traj.get_feature_for_rnn()

In [9]:
feats2traj.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 18, 10)            1080      
                                                                 
 dense_1 (Dense)             (None, 18, 3)             33        
                                                                 
Total params: 1,113
Trainable params: 1,113
Non-trainable params: 0
_________________________________________________________________


### <a id='toc1_3_2_'></a>[Entrainement du modèle](#toc0_)

In [10]:
feats2traj.fit(test_size=0.33, batch_size=100, epochs=10)

Epoch 1/10
67/67 [==============================] - 1s 8ms/step - loss: 1.1718 - val_loss: 0.7935
Epoch 2/10
67/67 [==============================] - 0s 5ms/step - loss: 0.5259 - val_loss: 0.4632
Epoch 3/10
67/67 [==============================] - 0s 5ms/step - loss: 0.3586 - val_loss: 0.3926
Epoch 4/10
67/67 [==============================] - 0s 5ms/step - loss: 0.3203 - val_loss: 0.3688
Epoch 5/10
67/67 [==============================] - 0s 5ms/step - loss: 0.3063 - val_loss: 0.3604
Epoch 6/10
67/67 [==============================] - 0s 5ms/step - loss: 0.3003 - val_loss: 0.3555
Epoch 7/10
67/67 [==============================] - 0s 6ms/step - loss: 0.2973 - val_loss: 0.3532
Epoch 8/10
67/67 [==============================] - 0s 6ms/step - loss: 0.2957 - val_loss: 0.3528
Epoch 9/10
67/67 [==============================] - 0s 6ms/step - loss: 0.2948 - val_loss: 0.3514
Epoch 10/10
67/67 [==============================] - 0s 5ms/step - loss: 0.2940 - val_loss: 0.3507


In [11]:
px.line(pd.DataFrame(feats2traj.history.history))

### <a id='toc1_3_3_'></a>[Exemple de prédiction](#toc0_)

In [31]:
trajectory_pred_lib, trajectory_true_lib = feats2traj.compare_pred_true(indiv=0)
print("Vérité :")
trajectory_true_lib


1/1 [==============================] - 0s 22ms/step
Vérité :


[('3', '7', array([5.])),
 ('3', '1', array([5.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.])),
 ('<pad>', '<pad>', array([0.]))]

In [32]:
print("Prédiction :")
trajectory_pred_lib

Prédiction


[('3', '8', 8.85818),
 ('3', '8', 6.799532),
 ('<pad>', '<pad>', 4.3625536),
 ('<pad>', '<pad>', 2.3071275),
 ('<pad>', '<pad>', 0.95579445),
 ('<pad>', '<pad>', 0.2500274),
 ('<pad>', '<pad>', -0.041931383),
 ('<pad>', '<pad>', -0.12851933),
 ('<pad>', '<pad>', -0.131903),
 ('<pad>', '<pad>', -0.10916446),
 ('<pad>', '<pad>', -0.08325182),
 ('<pad>', '<pad>', -0.06183087),
 ('<pad>', '<pad>', -0.046351917),
 ('<pad>', '<pad>', -0.03605597),
 ('<pad>', '<pad>', -0.029641822),
 ('<pad>', '<pad>', -0.025890995),
 ('<pad>', '<pad>', -0.023858305),
 ('<pad>', '<pad>', -0.022870993)]